In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# this is the EM specific package for querying the EM data
from caveclient import CAVEclient


from meshparty import trimesh_io, trimesh_vtk
from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)
# # # if you have not yet setup this computer, uncomment this below line
# # # paste the token from the website in, and run the line


In [ ]:
# client.auth.save_token(token="fca0294ed408d104e0eebe74514c744b", overwrite=True)

# # # then comment or delete the line as you don't need to run it on this computer  again

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
client.materialize.get_tables()

In [ ]:
client.materialize.get_versions()

# Create the pre_to_mn_df 
with index (segID,has_soma,sensory,neck,local) 
x 
(side, nerve,segment,function,muscle,rank,segID)

In [ ]:
pre_to_mn_df = connectome_create.left_pre_to_mn_df(client)
#pre_to_mn_df = connectome_create.load_pre_to_mn_df()

In [ ]:
# pre_to_mn_df.loc[pre_to_mn_df.pre_pt_root_id==648518346489965495]
(pre_to_mn_df.index.get_level_values('segID').to_numpy()==648518346489965495).any()
# Ha, not there anymore!


In [ ]:
import datetime as dt

ts = dt.datetime.utctimetuple(dt.datetime.utcnow())
my_tup = ts[0:3]
type(my_tup)
my_tup
dt.datetime(ts[0],ts[1],ts[2])

# Spot check some cells in the index

In [ ]:
idx_df = pre_to_mn_df.index.to_frame()
# # idx_df.loc[648518346515671370]  # should now have a soma
# # idx_df.loc[648518346509933350]  # should now have a soma
# idx_df.loc[648518346492640443]  # should now have a soma
# idx_df.loc[648518346514142507]  # should now have a soma
# idx_df.loc[648518346494279701]  # should now have a soma
# # All of the cells with bad somas are now in the soma table, 5/24/22
idx_df.shape

# Update the status of some fragments

In [ ]:
# Now find some segids from points

frags = pd.read_csv('~/Code/connectomics/connectomics/annotations_fragments/fragment_point_list_20220524.csv')
frags = frags.rename(columns={'cell body position, other notes':'note'})

frags_pt_series = frags['pt_position']
frags_pt_ndarray = np.ndarray([len(frags_pt_series),3])

pt_ints = []
for rw in range(len(frags_pt_series)):
    #print(i)
    i = frags_pt_series[rw]
    i = i.split('(')[0]
    i = i.split(')')[0].split(', ')    
    i = np.array([j for j in i if j != ''])
    pt_ints.append(list(map(int, i)))
    frags_pt_ndarray[rw,:] = i

# frags_pt_ndarray = frags_pt_ndarray.astype(int)

# Find segIDs from points
cv = cloudvolume.CloudVolume(client.info.get_datastack_info()['segmentation_source'], use_https=True)
segIDs = utils.segIDs_from_pts_service(frags_pt_ndarray,cv)
frags['pt_root_id'] = segIDs
frags = frags[['pt_position','pt_root_id','index_logical']]
utils.save_df_as_pickle(frags,'frag_df')

# idx_df.loc[648518346515671370]  # should now have a soma
# All of the cells with bad somas are now in the soma table
All = slice(None)
print('There are {} cells with somas and {} sensory cells'.format(
    idx_df.loc[(All,False,False,False,False,False),'has_soma'].sum(),
    idx_df.loc[(All,False,False,False,False,False),'sensory'].sum()))

for idx,row in frags.iterrows():
    if row.index_logical == 'has_soma':
        try:
            idx_df.loc[(row.pt_root_id,All,All,All,All,All),'has_soma'] = True
        except KeyError:
            print('{} is no longer a fragment'.format(row.pt_root_id))
        # print(idx_df.loc[row.pt_root_id,'has_soma'])
    elif row.index_logical == 'sensory':
        idx_df.loc[(row.pt_root_id,All,All,All,All,All),'sensory'] = True
        # print(idx_df.loc[row.pt_root_id,'sensory'])

print('There are now {} cells with somas and {} sensory cells'.format(
    idx_df.loc[(All,False,False,False,False,False),'has_soma'].sum(),
    idx_df.loc[(All,False,False,False,False,False),'sensory'].sum()))


In [ ]:
len(segIDs)


In [ ]:
frags.pt_root_id.values

In [ ]:
pre_to_mn_df.index = pd.MultiIndex.from_frame(idx_df)
connectome_create.save_pre_to_mn_df(pre_to_mn_df,ext='edited')

In [ ]:
pre_to_mn_df2 = connectome_create.load_pre_to_mn_df(ext='edited')
print(type(pre_to_mn_df2))
pre_to_mn_df2.loc[(row.pt_root_id,False,True,False,False,False),:]

# Test suite

In [ ]:
All = slice(None)

In [ ]:
# How many motor neurons are presynaptic to motor neurons
# segID	motor	has_soma	sensory	 neck	local	
motor = pre_to_mn_df.loc[(All,True,True,False,False,False),:]
# descending.index.get_level_values('segID').to_list()
motor.shape

In [ ]:
# How many sensory neurons # 
# segID	motor	has_soma	sensory	 neck	local	
sensory = pre_to_mn_df.loc[(All,False,False,True,False,False),:]
# descending.index.get_level_values('segID').to_list()
sensory.shape

In [ ]:
# How many weird neurons # 
# segID	motor	has_soma	sensory	 neck	local	
weird = pre_to_mn_df.loc[(All,False,True,True,False,False),:]
# descending.index.get_level_values('segID').to_list()
weird

In [ ]:
# How many descending neurons # 217
# segID	motor	has_soma	sensory	 neck	local	
descending = pre_to_mn_df.loc[(All,False,False,False,True),:]
# descending.index.get_level_values('segID').to_list()
descending.shape

In [ ]:
# How many motor neurons are presynaptic to motor neurons
# segID	motor	has_soma	sensory	 neck	local	
ascending = pre_to_mn_df.loc[(All,False,True,False,True,False),:]
# descending.index.get_level_values('segID').to_list()
ascending.head()
# some ascending that should be ascending
# 648518346507084872
# 648518346480786912
# 648518346490374748
sid = 648518346504729203
ascending.index.get_level_values('segID')==sid
# All are here

In [ ]:
# Fragments?
# segID	motor	has_soma	sensory	 neck	local	
fragments = pre_to_mn_df.loc[(All,False,False,False,False,False),:]
# descending.index.get_level_values('segID').to_list()
fragments.shape


In [ ]:
# Local and intersegmental?
# segID	motor	has_soma	sensory	 neck	local	
vncns = pre_to_mn_df.loc[(All,False,True,False,False,False),:]
# descending.index.get_level_values('segID').to_list()
vncns.shape

In [ ]:
# look for suspicious ids